In [40]:
import numpy as np
import csv
import sys
import json
import sys

points = sc.textFile('odm_prueba.xyz')
points_split = points.map(lambda x: (x.split(" ")))
points_parsed = points_split.map(lambda line: [float(line[0]), float(line[1]), float(line[2])])
cnt = points_parsed.take(5)
print cnt
print "\n"
points_enum = points_parsed.zipWithIndex().map(lambda x: (x[1],x[0]))
print points_enum.take(4)
print "\n"


# print "DSADASDAS"

# points_parsed_tuple = points_parsed.map(lambda x: ((x[0], x[1]), x[2]))
# print points_parsed_tuple.take(3)
# print "mapped (key, z) \n"

# # flags = points_parsed_tuple.filter(lambda (x,y):  all(y[1]))
# # print flags.take(5)
# # print "filter empty \n"

# flags = points_parsed_tuple.cogroup(points_parsed_tuple).map(lambda (x,y): (x,(list(y[0])[0],list(y[1])[0])) if(list(y[0])[0] != "" and list(y[1])[0] != "") else None)
# print flags.take(15)
# print "cogroup (key, (z_1, z_1)) \n"

# >= 0 <----- threshold
# flags = flags.map(lambda (key, tuple): 1 if(abs(tuple[0] - tuple[1]) >= 0) else 0)
# # flags = points_parsed_tuple.subtractByKey(dialation_p).map(lambda (key,z): 1 if( abs(z) > thres ) else 0)
# print flags.take(25)



# print "rwerwerew"

##################################################################

#np_pointcloud = np.array(pointcloud)

# Build different windows and height thresholds
windows = []
height_thresholds = []

# Default parameters
slope = 1.0
cell_size = 1.0
base = 2.0
max_distance = 1
initial_distance = 0.5
max_window_size = 20
window_type = 'linear'

window_size = 0.0
i = 0
while window_size < max_window_size:
    # Create different windows
    if window_type == 'linear':
        window_size = cell_size * ((2*(i+1)*base) + 1)
    elif window_type == 'exponential':
        window_size = cell_size * ((2*base**(i+1) + 1))
    if window_size > max_window_size:
        break
    windows.append(window_size)
    # Create corresponding height threshold
    if i == 0:
        height_threshold = initial_distance
    elif height_threshold > max_distance:
        height_threshold = max_distance
    else:
        height_threshold = slope*(windows[-1]-windows[-2])*cell_size + initial_distance


    height_thresholds.append(height_threshold)
    i += 1

for window in windows:
    print 'window size: ', window

for thresh in height_thresholds:
    print 'height thresholds: ', thresh

    
    
######################## MAKE CARTESIAN (KIND OF MATRIX points by points - all possible combinations) ##################

points_matrix = points_enum.cartesian(points_enum)
print 'result of cartesian'
print points_matrix.take(10)
neighbours = points_matrix.map(lambda (x,y): (x[0],y[0]) if ( y[1][0] >= x[1][0] - window_size/2 and y[1][0] <= x[1][0] + window_size/2 and y[1][1] >= x[1][1] - window_size/2 and y[1][1] <= x[1][1] + window_size/2 ) else None)
print 'neighbours'
print neighbours.take(14)
print "\n"   
print 'neighbour rdd finished'


######################## THE TUPLES ARE PAIRS OF NEIGHBOURS ############################################################
# PORABABLY WE DON'T NEED TO SORT ANYTHINGS AS WE'LL ITERATE OVER COMPLETE LISTs ANYWAY

print "\n"  
print "\n"  
print "\n"  
# ACUTALLY WE CAN JUST RETURN INDEX OF POINT x[0], INDEX OF NEIGHBOUR y[0] AND THE Z-VALUE OF ITS NEIGHBOUR Y[1][2]
neighbours = points_matrix.map(lambda (x,y): (x[0], (y[0], y[1][2])) if ( y[1][0] >= x[1][0] - window_size/2 and y[1][0] <= x[1][0] + window_size/2 and y[1][1] >= x[1][1] - window_size/2 and y[1][1] <= x[1][1] + window_size/2 ) else None)
zvals = points_matrix.map(lambda (x,y): (x[0], y[1][2]) if ( y[1][0] >= x[1][0] - window_size/2 and y[1][0] <= x[1][0] + window_size/2 and y[1][1] >= x[1][1] - window_size/2 and y[1][1] <= x[1][1] + window_size/2 ) else None)
new_points = points_matrix.map(lambda (x,y): ((x[1][0], x[1][1]), y[1][2]) if ( y[1][0] >= x[1][0] - window_size/2 and y[1][0] <= x[1][0] + window_size/2 and y[1][1] >= x[1][1] - window_size/2 and y[1][1] <= x[1][1] + window_size/2 ) else None)
print new_points.take(5)
print "\n"
print "New points - DONE"
print "\n"


erosion_p = new_points.filter(lambda x: x!= None).groupByKey().map(lambda (key,z): (key, list(z))).mapValues(min)
erosion_p = erosion_p.map(lambda (key,z): [key[0], key[1], z])
print erosion_p.take(10) 

print "\n"
print "Erosion - DONE"
print "\n"

points_enum_2 = erosion_p.zipWithIndex().map(lambda x: (x[1],x[0]))
points_matrix_2 = points_enum_2.cartesian(points_enum_2)
new_points_2 = points_matrix_2.map(lambda (x,y): ((x[1][0], x[1][1]), y[1][2]) if ( y[1][0] >= x[1][0] - window_size/2 and y[1][0] <= x[1][0] + window_size/2 and y[1][1] >= x[1][1] - window_size/2 and y[1][1] <= x[1][1] + window_size/2 ) else None)
dialation_p = new_points_2.filter(lambda x: x!= None).groupByKey().map(lambda (key,z): (key, list(z))).mapValues(max)
# dialation_p = dialation_p.map(lambda (key,z): (key[0], key[1], z))
print dialation_p.take(10) 
print "\n"
print "Dialation - DONE"
print "\n"


points_parsed_tuple = points_parsed.map(lambda x: ((x[0], x[1]), x[2]))
print points_parsed_tuple.take(3)

# flags = points_parsed_tuple.cogroup(dialation_p).filter(lambda (x,y):  all(y[1]))
# print flags.take(5)

flags = points_parsed_tuple.cogroup(dialation_p).map(lambda (x,y): (x,(list(y[0])[0],list(y[1])[0])) if(list(y[0])[0] != "" and list(y[1])[0] != "") else None)
print flags.take(15)

thres = 0
flags = flags.map(lambda (key, tuple): 1 if(abs(tuple[0] - tuple[1]) > thres) else 0)
# flags = points_parsed_tuple.subtractByKey(dialation_p).map(lambda (key,z): 1 if( abs(z) > thres ) else 0)
print flags.take(25)


sys.exit()

for window, thres in zip(windows, height_thresholds):

    for point_idx in range(np_pointcloud.shape[0]):
        if ( abs(np_pointcloud[point_idx,-1] - pointcloud_copy[point_idx,-1])) > thres:
            flags[point_idx] = 1


with open('dataset/pcloud1.xyz', 'wb') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=' ')
    csvwriter.writerows(np_pointcloud[np.where(flags == 1)[0], :].tolist())

with open('dataset/pcloud2.xyz', 'wb') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=' ')
    csvwriter.writerows(np_pointcloud[np.where(flags == 0)[0], :].tolist())

[[-255.509995, -55.193001, -154.910004], [-255.460007, -55.193001, -154.919998], [-264.429993, -55.193001, -154.699997], [-262.190002, -55.193001, -154.759995], [-259.950012, -55.193001, -154.850006]]


[(0, [-255.509995, -55.193001, -154.910004]), (1, [-255.460007, -55.193001, -154.919998]), (2, [-264.429993, -55.193001, -154.699997]), (3, [-262.190002, -55.193001, -154.759995])]


window size:  5.0
window size:  9.0
window size:  13.0
window size:  17.0
height thresholds:  0.5
height thresholds:  4.5
height thresholds:  1
height thresholds:  4.5
result of cartesian
[((0, [-255.509995, -55.193001, -154.910004]), (0, [-255.509995, -55.193001, -154.910004])), ((0, [-255.509995, -55.193001, -154.910004]), (1, [-255.460007, -55.193001, -154.919998])), ((0, [-255.509995, -55.193001, -154.910004]), (2, [-264.429993, -55.193001, -154.699997])), ((0, [-255.509995, -55.193001, -154.910004]), (3, [-262.190002, -55.193001, -154.759995])), ((0, [-255.509995, -55.193001, -154.910004]), (4, [-259.9

SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.
